### Create environment

In [1]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

import gym_kilobots

In [2]:
# Define four objects in a centered square
square_objects = [
    ((-0.25,  0.25), 0.0),   # top-left
    (( 0.25,  0.25), 0.0),   # top-right
    (( 0.25, -0.25), 0.0),   # bottom-right
    ((-0.25, -0.25), 0.0),   # bottom-left
]

# Define 1 object in the center of the environment
single_center_object = [
    ((0.0, 0.0), 0.0)        # center,
]

# Choose a fixed light position
light_pos = (-0.1, 0.1)

# Choose four explicit kilobot positions
kb_positions = [
        (-0.10, 0.20),
        (-0.10, 0.20),
        (-0.20, 0.10),
        (-0.20, 0.10),
    ]

# NOTE: if you don't do this, the environment will randomly place the objects
env = gym.make(
    'Kilobots-QuadAssembly-v0',
    render_mode='human',
    num_kilobots=len(kb_positions),
    object_config=single_center_object,
    light_position=light_pos,
    kilobot_positions=kb_positions
)


In [3]:
check_env(env,warn=True)

/opt/anaconda3/envs/swarm/lib/python3.13/site-packages/stable_baselines3/common/env_checker.py:462: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


In [ ]:
#vec_env = make_vec_env(env, n_envs = 1)

# Train the agent
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=500000)
model.save("swarm_ppo")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 474  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 437           |
|    iterations           | 2             |
|    time_elapsed         | 9             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 3.7327118e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.83         |
|    explained_variance   | 0.00089       |
|    learning_rate        | 0.0003        |
|    loss                 | 1.89e+04      |
|    n_updates            | 10            |
|    policy_gradient_loss | -8.55e-0

In [ ]:
model = PPO.load("swarm_ppo")

def controller(x):
      action,states = model.predict(x)
      return action

running = True
obs, info = env.reset()

while running:

      env.render()

      x = obs
      action = controller(x)
      obs, reward, terminated, truncated, info = env.step(action)

      print('Reward: ', reward)

      # Print the orientation of the objects using the unwrapped environment
      #print(env.unwrapped.get_objects_status())

      # End episode
      if terminated or truncated:
            break

env.close()  # Ensure the environment is properly closed